In this lab we will study the initial approaches in the context of multiagent systems based on game theory. Mainly the concept of Nash equilibria including repeated games.


# Installation
For this lab we will only need numpy. The toolbox nashpy has already several algorithms implemented and allows to verify some of the results.

In [ ]:
#nashpy toolbox
!pip install nashpy
!pip install matplotlib
#Knight, V., & Campbell, J. (2018). Nashpy: A Python library for the computation of Nash equilibria. Journal of Open Source Software, 3(30), 904.
#https://nashpy.readthedocs.io/en/stable/text-book/index.html
#https://nashpy.readthedocs.io/en/stable/

import nashpy as nash
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# @title Define Games
#
# We can use the toolbox to define some standard games.
import warnings

Games = {}
A = np.array([[1, -1], [-1, 1]])
Games['Matching_pennies'] = nash.Game(A)

A = np.array([[-1, 1], [1, -1]])
Games['UnMatching_pennies'] = nash.Game(A)

A = np.array([[1, 3], [0, 2]])
B = np.array([[1, 0], [3, 2]])
Games['Prisioner_dilema'] = nash.Game(A,B)

A = np.array([[-2, -2], [6, 0]])
B = np.array([[0, 6], [3, 3]])
Games['Hawk-Dove'] = nash.Game(A,B)

A = np.array([[0,-1,1],[1,0,-1],[-1,1,0]])
Games['Rock-Paper-Scissor'] = nash.Game(A)


# Activity 1 - Nash equilibria
For each of the previous games verify if a Nash equilibria exists in pure strategies.


In [ ]:
#Compute pure Nash equilibria
def findPureNash(g):
    nasheq = []
    #TODO
    pass

    return nasheq

for kk in Games.keys():
    g = Games[kk]
    ret = findPureNash(g)
    if ret:
        print(kk, "Pure nash equilibria  ", ret)
    else:
        print(kk, "has no pure NE")



You can verify your solution with the following code, that uses the various methods of the toolbox, to compute the Nash equilibria.

In [ ]:
# game solutions based on Nashpy

for kk in Games.keys():
    g = Games[kk]

    if g.zero_sum:
        a = g.linear_program()
    else:
        a = g.vertex_enumeration()
        #b = g.support_enumeration()
    print(kk,"\n Nash equilibria\n",list(a))

    # Ficticious Play
    np.random.seed(0)
    iterations = 5000
    play_counts = g.fictitious_play(iterations=iterations)
    play_counts=np.array(list(play_counts))
    print(" ficticious play\n", play_counts[-1,:,:]/(play_counts.shape[0]-1))


# Activity 2
Nash equilibria is in many cases not the best for both players and other strategies have been proposed. Define 3 strategies, always Nash, Tit-for-Tat (repeat the action of the other player), or Tit-for-Two-Tats (defect only after two defects). Make games between all the types of agents. What is the best strategy?

In [ ]:
strat = ['nash','t4t','t42t']
N = 100
g = Games['Prisioner_dilema']

# player - player number 0/1
# P - list with the previous plays
# strat - strategy of the player ['nash','t4t','t42t']
# it should return a pure strategy as a vector, e.g. [1,0] to choose the first action
def play( player, P, strat):
    #TODO
    pass

    return st


for strat0 in strat:
    for strat1 in strat:
        P = [[np.random.randint(0,2),np.random.randint(0,2)]]
        R = []
        for nn in range(0,N):
            p0 = play( 0, P, strat0)
            p1 = play( 1, P, strat1)
            P.append([p0.index(1),p1.index(1)])
            r = g[p0,p1]
            R.append([r[0],r[1]])
        print(strat0,strat1,np.sum(np.array(R),axis=0))

# Questions
Is playing the NE always better?

Are these strategies meaningfull in all games? How do the different strategies result in the different games?

Which of the previous strategies is more robust to the shaking hand problem, i.e. when an agent wants to make an action but - by mistake - does another? How can you change the code to verify it?

# Activity 3
Similar to game theory, Evolutionary stable strategies explain how the strategy of a complete species can evolve. The `g.replicator_dynamics` function for the toolbox can compute this as in this code.
Try to replicate the code in the following cell.
Answer the following questions. Which games have an ESS? How does it related with the existing Nash Equilibria? What are the properties of the games where there is not an ESS?

[Evolutionary stable strategies](https://nashpy.readthedocs.io/en/v0.0.22/reference/replicator-dynamics.html)

In [ ]:
# @title ESS
import matplotlib.pyplot as plt

g = Games['Rock-Paper-Scissor']
y0 = np.array([0.5, 0.3, 0.2])

nA = g.payoff_matrices[0].shape[0]

mut = 0.0
mutation = np.eye(nA)*(1-mut)+(1-np.eye(nA))*mut/(nA-1)

play_counts = g.replicator_dynamics(y0=y0, timepoints=np.linspace(0, 10, 100)
                                        ,mutation_matrix=mutation)
print(play_counts[-1,:])
plt.plot(play_counts)
plt.legend(['Rock','Paper','Scissor'])


g = Games['Prisioner_dilema']
y0 = np.array([0.1,0.9])

nA = g.payoff_matrices[0].shape[0]

mut = 0.0
mutation = np.eye(nA)*(1-mut)+(1-np.eye(nA))*mut/(nA-1)

play_counts = g.replicator_dynamics(y0=y0, timepoints=np.linspace(0, 10, 100)
                                        ,mutation_matrix=mutation)
print(play_counts[-1,:])
plt.figure()
plt.plot(play_counts)
plt.legend(['Defect','Cooperate'])


In the following cell we compute explicitly the dynamics.

In [ ]:
g = Games['Prisioner_dilema']
g = Games['UnMatching_pennies']
g = Games['Rock-Paper-Scissor']

th = np.array([0.5, 0.3, 0.2])
dt = 0.01
TH = []

def dynreplicator(g,th):
    # todo
    va = g.payoff_matrices[0]@th
    v = th@g.payoff_matrices[0]@th

    dth = th * (va-v)

    return dth

for ii in np.arange(0,20,dt):

    dth = dynreplicator(g,th)
    th = np.clip(th + dth * dt, 0, 1)
    th /= np.sum(th)
    TH += [th]
    #print(th,va,v,dt)
    #plt.figure()
plt.plot(TH)
plt.legend(['Rock','Paper','Scissor'])